In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies
 , UnicodeSyntax, ScopedTypeVariables, NoMonomorphismRestriction #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Manifold.Shade
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Math.LinearMap.Category.Derivatives
import Data.VectorSpace
import Data.Function.Affine
import Data.Basis (Basis)
import Linear(V2(..), ex, ey, _x, _y)
import Data.Semigroup
import Data.List.NonEmpty (NonEmpty(..))
import qualified Data.Foldable as Hask
import qualified Control.Monad as Hask
import qualified Control.Comonad.Cofree as Hask
import Control.Lens
:opt no-lint
import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
test x = x `seq` putStrLn "ok"

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices, Point(P))

In [3]:
type X = ℝ
type T = ℝ
type U = ℝ
type Ðx'U = ℝ
type Ðx²'U = ℝ
type x × y = ℝ²
type HeatFlow = (U, Ðx'U)
et = ey :: Basis ℝ²
_t = _y :: Lens' (X × T) T
_u = _1 :: Lens' (U, Ðx'U) U
_ðx'u = _2 :: Lens' (U, Ðx'U) Ðx'U

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [4]:
import Graphics.Dynamic.Plot.R2
import Data.Colour.Names
import Data.Colour.Manifold

colourscheme :: Shade' ℝ -> Shade (Colour ℝ)
colourscheme (Shade' u du) = interp (Shade u $ dualNorm du :: Shade ℝ)
 where Just interp = rangeOnGeodesic darkblue orange
 
prettyWebPlot :: PointsWeb ℝ² y -> DynamicPlottable
prettyWebPlot w = plot [ diagramPlot . opacity 0.5 $ fromVertices [P r₁, P r₂]
                       | ((r₁@(V2 x₁ y₁),_),(r₂@(V2 x₂ y₂),_)) <- edg ]
 where edg = webEdges w
 
colourscheme_heat :: Shade' HeatFlow -> Shade (Colour ℝ)
colourscheme_heat = f `seq` \(Shade' y e) -> f (Shade y $ dualNorm e)
 where Just (f :: Shade HeatFlow -> Shade (Colour ℝ))
          = rangeWithinVertices ((0,0), neutral)
                              [ ((1,0), orange)
                              , ((0,2), seagreen) ]
       Just neutral = toInterior (dimgrey :: Colour ℝ)
test colourscheme_heat

ok

In [5]:
trimDecimals :: ℝ -> ℝ
trimDecimals = (/q) . fromInteger . round . (*q)
 where q = 100

import Control.Applicative ((<|>))
import Control.Monad.Trans.Except
honestStrategy, cheatingStrategy :: ∀ x y. (Refinable y, y ~ Interior y)
   => InformationMergeStrategy [] (Except (PropagationInconsistency x (Shade' y))) (x, Shade' y) (Shade' y)
honestStrategy = indicateInconsistencies intersectShade's
cheatingStrategy = indicateInconsistencies (\l -> intersectShade's l <|> mixShade's l)

In [6]:
α = 0.5
(xl,xr) = (0,2)

exact_heat :: X -> T -> U
exact_heat x t = u 1 - u 3/3
 where u n = sin (n*pi*(x-xl)/l) * exp(-α * (n*pi/l)^2 * t)
       l = xr - xl

plotWindow [ continFnPlot (`exact_heat`t) & legendName ("t = "++show t)
           | t<-[0,0.2 .. 1] ]

GraphWindowSpecR2{lBound=-1.3333333333333335, rBound=1.3333333333333335, bBound=-1.529313244967971, tBound=1.5293132449679738, xResolution=640, yResolution=480}

In [7]:
δx₀ = 1/12
δt₀ = 7/8 * δx₀

heatEqn :: DifferentialEqn (X × T) Ðx²'U U
heatEqn (Shade (V2 x t, _) _) = LocalDifferentialEqn {
     _predictDerivatives = \sh_ðx²'u
       -> return $ embedShade (lensEmbedding (α*∂id/∂_t)) sh_ðx²'u
   , _rescanDerivatives = \d⁰ d¹ d²
       -> ( return $ if abs t<δt₀/2 || min (abs x) (abs $ xr-x) < δx₀/2
                          then exact_heat x t|±|[5e-2]
                          else d⁰
          , return $ projectShade (lensEmbedding (1*∂id/∂_x.∂_x)) d² )
   }

In [8]:
startSt_heat = fromWebNodes euclideanMetric
          [ (V2 x t, exact_heat x t|±|[1])
          | t'<-trimDecimals<$>[tStart, tStart+δt₀ .. tEnd]
          , let xlq = xl + skew*(t' - tEnd)
          , x<-trimDecimals<$>[xlq-δx₀, xlq .. xr+δx₀]
          , x >= xl-δx₀
          , let t = t' + 0.005 * sin (37245*sin(2334*x)+t') ]
 where skew = 8/14
       xlq = -1
       (tStart, tEnd) = (-δt₀, 1)


tfs_heat = iterateFilterDEqn_static honestStrategy id heatEqn startSt_heat
forM_ [Hask.toList tfs_heat ]
  $ \tfs ->
    plotWindow
       [-- prettyWebPlot $ head tfs
        plotLatest [ plot (fmap colourscheme tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ]
       , dynamicAxes ]
Hask.mapM_ (\st -> mapM_ (\((p,y),(q,z)) -> putStrLn $
                 show p ++ ":\t" ++ prettyShowShade' y
            ++ "\n" ++ show q ++ ":\t" ++ prettyShowShade' z) (webEdges st) >> print())
    $ take 0 (Hask.toList tfs_heat)

In [ ]:
pertubation :: X -> T -> U
pertubation x t = 0.4 * exp (-((x-0.7)^2 + (t-0.6)^2)*3)

startSt_perturbed = fromWebNodes euclideanMetric
          [ (V2 x t, (exact_heat x t + pertubation x t, 0)
                       |±|[(0.01,0), (0,0.10)])
          | x<-[xl,xl+δx₀ .. xr], t<-[tStart, tStart+δt₀ .. tEnd]]
 where δx₀ = 0.0625
       δt₀ = 0.0625

tfs_heat = iterateFilterDEqn_static strategy id heatEqn startSt_perturbed
forM_ [Hask.toList tfs_heat ]
  $ \tfs ->
    plotWindow
       [ prettyWebPlot $ head tfs
       , plotLatest [ plot (fmap colourscheme_heat tfi)
                       & legendName ("i = "++show i)
                    | (i,tfi) <- zip [0..] tfs ] ]
Hask.mapM_ (\st -> mapM_ (\((p,y),(q,z)) -> putStrLn $
                 show p ++ ":\t" ++ prettyShowShade' y
            ++ "\n" ++ show q ++ ":\t" ++ prettyShowShade' z) (webEdges st) >> print())
    $ take 0 (Hask.toList tfs_heat)

In [16]:
import Control.Monad.Trans.Except
findErr :: (Hask.Monad m, Hask.Foldable m) => Hask.Cofree m a -> m ()
findErr (a Hask.:< q) = case Hask.toList q of
        [] -> const () <$> q
        l  -> foldr1 (>>) $ findErr<$>l

case runExcept $ findErr tfs_heat of
 Left (PropagationInconsistency pro apr) -> do
      putStrLn $ "apriori: "++show apr
      forM_ pro $ \(o,m) -> putStrLn $ show o ++ "\t-> " ++ show m
      --plotWindow $ (plot apr & legendName "apriori")
        --         : [plot m & legendName ("from "++show o) | (o,m)<-pro]


apriori: 0.49|±|[0.05]
V2 0.55 7.0e-2	-> 0|±|[18]
V2 0.47 7.0e-2	-> -0|±|[22]
V2 0.6 0.0	-> 1|±|[471]
V2 0.43 0.0	-> 0|±|[]
V2 0.47 (-7.0e-2)	-> 2|±|[19]
V2 0.56 (-7.0e-2)	-> 162|±|[80]

![Initial state from which to start refining solution of the heat PDE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/PDE-solution-filter/HeatEqn-InitialState.png)

In [21]:
intersectShade's $ 0|±|[18] :| [-0|±|[22], 1|±|[471]] :: Maybe (Shade' ℝ)

Just (0|±|[18])